# Ex 10: The moment of inertia tensor of a Space Station

Let's assume a space station define by the given modules:

Module|Mass|x|y|z
--|--|--|--|--
Astrophysics module|3|6|-2|0
Robotics repair plant|4|3|1|-2
Power plant|7|-4|1|2
Docking module|5|-4|-2|-4
Communications module|4|3|2|-1
Solar arrays|3|2|-1|6

We can then calculate the total mass, center of mass and inertia matrix of the station.

In [45]:
import numpy as np
from numpy.linalg import eig

modules = np.array([[3,  6, -2,  0],
                    [4,  3,  1, -2],
                    [7, -4,  1,  2],
                    [5, -4, -2, -4],
                    [4,  3,  2, -1],
                    [3,  2, -1,  6]])

total_mass = np.sum(modules, axis=0)[0]
total_mass

26

The center of mass of the space station can be find using the mass and position off all the submodules:

$$x_{com}=\frac{\sum{m_i*x_i}}{m_{total}}$$
$$y_{com}=\frac{\sum{m_i*y_i}}{m_{total}}$$
$$z_{com}=\frac{\sum{m_i*z_i}}{m_{total}}$$

In [46]:
center_of_mass = (modules[:, 1:].transpose() * modules[:, 0]).sum(axis=1) / total_mass
center_of_mass

array([0., 0., 0.])

Center of mass is at the origin of the coordinate system.

We can find the inertia matrix of the system with:

$$
I=
\left(
\begin{array}{ccc}
I_{xx} & I_{xy} & I_{xz} \\
I_{xy} & I_{yy} & I_{yz} \\
I_{xz} & I_{yz} & I_{zz} \\
\end{array}
\right)
=
\left(
\begin{array}{ccc}
\sum{m(y^2 + z^2)} & -\sum{mxy}         & -\sum{mxz} \\
-\sum{mxy}         & \sum{m(x^2 + z^2)} & -\sum{myz} \\
-\sum{mxz}         & -\sum{myz}         & \sum{m(x^2 + y^2)} \\
\end{array}
\right)
$$

In [47]:
def inertia_moment(mass, x, y):
    return mass * (x**2 + y**2)
def inertia_product(mass, x, y):
    return mass * x * y

inertia = np.array([[0, 0, 0],
                    [0, 0, 0],
                    [0, 0, 0]])

for m, x, y, z in modules:
    inertia += np.array([[inertia_moment(m, y, z), -inertia_product(m, x, y), -inertia_product(m, x, z)],
                         [-inertia_product(m, x, y), inertia_moment(m, x, z), -inertia_product(m, y, z)],
                         [-inertia_product(m, x, z), -inertia_product(m, y, z), inertia_moment(m, x, y)]])

inertia

array([[298,  -6, -24],
       [ -6, 620, -20],
       [-24, -20, 446]])

All symmetrical inertia matrices can be diagonalised using eig vectors as the new base and the moments of inertia for these primary directions will be the corresponding eig values.

In [48]:
w,v=eig(inertia)
print('E-value:', w)
print('E-vector', v)

E-value: [293.95245163 447.74525937 622.302289  ]
E-vector [[-0.98681235 -0.16155017  0.01014485]
 [-0.02793964  0.10826326 -0.99372956]
 [-0.15943887  0.98090805  0.11134918]]


## New Solar arrays mass

Now, let's update the modules tables with the below values and find the new center of mass and inertia matrix.

Module|Mass|x|y|z
--|--|--|--|--
Astrophysics module|3|6|-2|0
Robotics repair plant|4|3|1|-2
Power plant|7|-4|1|2
Docking module|5|-4|-2|-4
Communications module|4|3|2|-1
Solar arrays|5|2|-1|6

In [49]:
modules = np.array([[3,  6, -2,  0],
                    [4,  3,  1, -2],
                    [7, -4,  1,  2],
                    [5, -4, -2, -4],
                    [4,  3,  2, -1],
                    [5,  2, -1,  6]])

total_mass = np.sum(modules, axis=0)[0]
total_mass

28

In [50]:
center_of_mass = (modules[:, 1:].transpose() * modules[:, 0]).sum(axis=1) / total_mass
center_of_mass

array([ 0.14285714, -0.07142857,  0.42857143])

In [51]:
inertia = np.array([[0, 0, 0],
                    [0, 0, 0],
                    [0, 0, 0]], dtype=float)

for m, x, y, z in modules:
    inertia += np.array([[inertia_moment(m, y-center_of_mass[1], z-center_of_mass[2]), -inertia_product(m, x-center_of_mass[0], y-center_of_mass[1]), -inertia_product(m, x-center_of_mass[0], z-center_of_mass[2])],
                        [-inertia_product(m, x-center_of_mass[0], y-center_of_mass[1]), inertia_moment(m, x-center_of_mass[0], z-center_of_mass[2]), -inertia_product(m, y-center_of_mass[1], z-center_of_mass[2])],
                        [-inertia_product(m, x-center_of_mass[0], z-center_of_mass[2]), -inertia_product(m, y-center_of_mass[1], z-center_of_mass[2]), inertia_moment(m, x-center_of_mass[0], y-center_of_mass[1])]])

inertia

array([[366.71428571,  -2.28571429, -46.28571429],
       [ -2.28571429, 694.28571429,  -8.85714286],
       [-46.28571429,  -8.85714286, 455.28571429]])

In [52]:
w_new,v_new=eig(inertia)
print('E-value:', w)
print('E-vector', v)

E-value: [293.95245163 447.74525937 622.302289  ]
E-vector [[-0.98681235 -0.16155017  0.01014485]
 [-0.02793964  0.10826326 -0.99372956]
 [-0.15943887  0.98090805  0.11134918]]


In [54]:
w_new - w

array([52.89849109, 27.07498192, 72.31224128])

As expected the moment of inertia increase as the mass of the solar arrays increase because the station is now harder to rotate (therefore more stable to external perturbation). But we can acknowledge that a small change of mass impact drasticaly the inertia of the whole system.